In [268]:
#Import modules
import numpy as np

class Ring:
    def __init__(self, size):
        self.size = size
        self.ring = []
        self.sl = []

    def __str__(self):
        return str(self.ring)
        
    def addFilament(self, length):
        filPos = np.array(np.zeros(self.size), dtype = bool)
        start = np.random.randint(self.size)
        end = start + length
        if end > len(filPos):
            filPos[start:len(filPos)] = True
            filPos[0:end-len(filPos)] = True
        else:
            filPos[start:end] = True
        self.ring.append(filPos)
        return 1
       
    def sliceIt(self, pos, thickness):
        ring = np.array(self.ring)
        start = pos
        end = start + thickness
        if end > self.size:
            endPart = ring[:, start:self.size]
            startPart = ring[:, 0:end - self.size]
            sl = np.hstack((endPart,startPart))
        else:
            sl = ring[:, start:end]
        
        self.sl = sl
        
    def reportStats(self, verbose=True):
        sl = self.sl
        fils, thickness = sl.shape
        filament = np.array(np.ones(thickness), dtype=bool)
        nofilament = np.array(np.zeros(thickness), dtype=bool)
        completeFils = 0
        ends = 0
        for i in range(fils):
            if np.array_equal(sl[i], filament):
                completeFils += 1
            elif not np.array_equal(sl[i], nofilament):
                ends += 1
        if verbose:
            print('Numer of true filaments in the ring: {}'.format(len(self.ring)))
            print('Numer of filaments throught slice: {}'.format(completeFils))
            print('Numer of filaments ending in the slice: {}'.format(ends))
        return completeFils, ends

def simulate(ringSize, numFils, filLength):
    R = Ring(ringSize)
    for i in range(numFils):
        R.addFilament(filLength)
    return R

def makeStats(R, thickness, trials, verbose = False):
    endStat = []
    fils = []
    starts = []
    for i in range(trials):
        start = np.random.randint(R.size)
        R.sliceIt(start,thickness)
        fil, ends = R.reportStats(False)
        endStat.append(ends)
        fils.append(fil)
        starts.append(start)
        if verbose:
            print('Sliced at: {} - {}'.format(start, R.sl))

    endStat = np.array(endStat)
    fils = np.array(fils)

    print('Results:')
    print('Number of ends (simul): {}'.format(endStat.mean()))
    print('Number of observed filaments: {}'.format(fils.mean()))


In [283]:
D = 10
Nf = 1
L = 5
S = 3
Ne = 2 * Nf * (S - 1) / D   #4 * Nf * D / ( D - S )
No = Nf * (L + 1 - S) / D
print('Number of ends (theory) : {}'.format(Ne))
print('Number of observed filaments (theory) : {}'.format(No))
R = simulate(D, Nf, L)

Number of ends (theory) : 0.4
Number of observed filaments (theory) : 0.3


In [284]:
makeStats(R, S, 10000)

Results:
Number of ends (simul): 0.3944
Number of observed filaments: 0.3068


In [272]:
print(R)

[array([ True, False, False, False, False, False, False,  True,  True,  True], dtype=bool)]


## How does it work?


Let's say there is a container of *D*=10 positions and a filament with length *L*=5 randomly placed in the container. In this example, the filament starts at the position 3 of the container:

```
Container                 = |---|---|---|---|---|---|---|---|---|---|
Filament                  =         |---|---|---|---|---|
```

Let's now place the filament in the container and for that, we will mark the positions in the container with the filament with T and without the filament with F.

```
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-|
```

Now, let's suppose that we can only look at *S*=3 positions at once in this container, which we will call *slice*. The question is:

**What is the probability of finding one of the ends of the filament?**

Probabilities is, in general, the ration between the number of possibilities of what you want and the number of *all* possibilities.

Because the container is "connected" from the end to the begining, the number of *all* possible slices, independently of the number of slice and it is the length of the the container *D*.

Now, let's consider that we see an end when there is a switch from **T** to **F** or **F** to **T** within the 2 positions we are allowed to look at it. In this example the two ends of the filament are in the position 3 and position 7. Let's count how many possible slices os *S*=3 can capture at least one of them:

|slice position|pos 1 | pos 2 | pos 3 | end |
|:-:|:-: |:-:|:-:|
| 1 | F |F| T| Yes|
| 2 | F |T| T| Yes|
| 3 | T |T| T| No |
| 4 | T |T| T| No |
| 5 | T |T| T| No |
| 6 | T |T| F| Yes|
| 7 | T |F| F| Yes|
| 8 | F |F| F| No |
| 9 | F |F| F| No |
| 10| F |F| F| No |

There are two possible slices, the slice at 1 and 2, related to the left terminus of the filament and the two others, slices 6 and 7, related to the right terminus. 
Thus, the probability of finding one end in a slice of 3 positions is: $4/10$ = 0.25. Notice that this is the contribution of 2 positions for each end.

For the sake of clarity, let's increase the size of the slice to 4. *S*=4:

|slice position|pos 1 | pos 2 | pos 3 | pos 4 | end |
|:-:|:-: |:-:|:-:|:-:|
| 1 | F |F| T| T| Yes|
| 2 | F |T| T| T| Yes|
| 3 | T |T| T| T| No |
| 4 | T |T| T| T| No |
| 5 | T |T| T| F| Yes |
| 6 | T |T| F| F| Yes|
| 7 | T |F| F| F| Yes|
| 8 | F |F| F| F| No |
| 9 | F |F| F| F| No |
| 10| F |F| F| T| Yes |

Now we have three possible slices for the left terminus:10, 1 and 2. Also we have three spossible slices for the right terminus: and 5,6 and 7. Which gives us 6 possibilities and thus, the probability of finding one end in a slice of 4 positions is: $6/10$ = 0.25. Notice that from the tota of 6 possibilities, 3 comes from each end.

Based on these results in this discrete example, it is easily to see that the number of possible slices that contain one of the size of the slice minus 1 for each end. Just to make it even more clear... let's look at one of the ends:

```
Position:                     1   2   3   4   5   6   7   8   9   10   1
Container with filament:  = |-F-|-F-|-T-|-T-|-T-|-T-|-T-|-F-|-F-|-F-||-F-|
slice 7 - no end          =             |---|---|---|---|
slice 5                   =                 |---|---|---|---|
slice 6                   =                     |---|---|---|---|
slice 7                   =                         |---|---|---|---|
slice 8 - no end          =                             |---|---|---||---|
```

Finally, we can confidently say that the probability of finding one of the ends of the filament of length *L* randomnly placed in a container of size *D* with a slice of size *S* is:

$$ P_{end} = 2 * (S - 1) / D $$

where the **2** is because there are two ends in a filament.


To generalize this to more than one filament, and assuming that their placement do not affect the placement of other filaments, one just need to multiply it by the number of filaments




*This assumes that $S << L << D$



$$N_{fil} * 2 = N_{ends}$$
$$ <N_{ends}>_{per slice} = N_{ends} . S_{thick} / D_{ring} = 2 * N_{fil} S_{thick} / D_{ring}$$

In [143]:
2*500*20/2000

10.0

In [147]:
b = np.array(1)

In [100]:
b.

array([False, False, False], dtype=bool)

In [103]:
np.array_equal(R.sl[1], nofilament)

False